In [8]:
import glob,os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

DATADIR = "C:\\Users\\Aaron\\Documents\\data\\planes\\"

CATEGORIES = ["drone", "fighter-jet", "helicopter", "missile", "passenger-plane", "rocket"]

img_array = []

for category in CATEGORIES:  # do all categories
    path = os.path.join(DATADIR,category)  # create path to each category
    for img in os.listdir(path):  # iterate over each image per each category
        img_array.append(cv2.imread(os.path.join(path,img) ,cv2.IMREAD_GRAYSCALE))  # convert to array

In [17]:
training_data = []
IMG_SIZE = 60
def create_training_data():
    for category in CATEGORIES:  # do all 6 categories

        path = os.path.join(DATADIR,category)  # create path to all 6 classes
        class_num = CATEGORIES.index(category)  # get the classification number for each class

        for img in tqdm(os.listdir(path)):  # iterate over each image per dogs and cats
            try:
                img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_GRAYSCALE)  # convert to array
                data_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize to normalize data size
                training_data.append([data_array, class_num])  # add this to our training_data
            except Exception as e:  # in the interest in keeping the output clean...
                pass
            
create_training_data()

print(len(training_data))
# Our dataset now has 8509 total samples

100%|██████████| 1186/1186 [00:14<00:00, 83.36it/s]

8509


In [29]:
import random
# shuffle our training data
random.shuffle(training_data)

In [30]:
for sample in training_data[:10]:
    print(sample[1])
# Just a quick peek on our shuffled data
# The reason we do this is so that our algorithm can properly learn the images, if they were sequential the 
# the model would get easily confused and have incorrect weights and might not reach optimality

0
2
2
3
3
1
1
2
3
4


In [31]:
X = []
y = []

for features,label in training_data:
    X.append(features)
    y.append(label)

print(X[0].reshape(-1, IMG_SIZE, IMG_SIZE, 1))

X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)

[[[[193]
   [201]
   [190]
   ...
   [ 58]
   [ 61]
   [ 56]]

  [[179]
   [ 83]
   [200]
   ...
   [ 59]
   [ 61]
   [ 63]]

  [[136]
   [159]
   [109]
   ...
   [104]
   [155]
   [237]]

  ...

  [[ 41]
   [ 42]
   [ 99]
   ...
   [ 83]
   [ 92]
   [ 96]]

  [[ 43]
   [ 44]
   [ 45]
   ...
   [102]
   [ 87]
   [ 92]]

  [[ 48]
   [ 44]
   [ 43]
   ...
   [187]
   [ 76]
   [ 87]]]]


In [38]:
y

[0,
 2,
 2,
 3,
 3,
 1,
 1,
 2,
 3,
 4,
 1,
 2,
 2,
 0,
 4,
 3,
 0,
 0,
 0,
 0,
 1,
 1,
 2,
 1,
 2,
 5,
 4,
 2,
 1,
 4,
 3,
 2,
 2,
 3,
 1,
 0,
 4,
 1,
 1,
 4,
 4,
 1,
 2,
 5,
 0,
 5,
 1,
 1,
 2,
 4,
 4,
 0,
 5,
 2,
 2,
 3,
 4,
 0,
 1,
 0,
 3,
 0,
 3,
 3,
 5,
 3,
 0,
 4,
 4,
 4,
 5,
 3,
 2,
 3,
 4,
 4,
 2,
 1,
 2,
 0,
 1,
 1,
 5,
 5,
 1,
 4,
 0,
 1,
 5,
 0,
 1,
 3,
 2,
 1,
 3,
 1,
 4,
 2,
 5,
 1,
 1,
 4,
 5,
 1,
 4,
 3,
 2,
 5,
 4,
 0,
 0,
 5,
 3,
 3,
 2,
 2,
 3,
 1,
 0,
 3,
 4,
 5,
 0,
 0,
 4,
 1,
 3,
 2,
 4,
 3,
 4,
 2,
 1,
 4,
 1,
 4,
 1,
 0,
 1,
 3,
 4,
 1,
 2,
 1,
 1,
 5,
 4,
 1,
 4,
 4,
 5,
 1,
 1,
 3,
 2,
 3,
 1,
 3,
 3,
 4,
 1,
 2,
 1,
 4,
 3,
 5,
 0,
 1,
 2,
 4,
 3,
 1,
 5,
 2,
 0,
 2,
 2,
 3,
 2,
 5,
 2,
 3,
 0,
 3,
 3,
 1,
 2,
 0,
 3,
 0,
 3,
 4,
 5,
 3,
 5,
 2,
 2,
 2,
 4,
 5,
 4,
 1,
 5,
 1,
 2,
 2,
 5,
 4,
 0,
 5,
 1,
 4,
 0,
 2,
 3,
 1,
 3,
 1,
 2,
 3,
 2,
 4,
 4,
 5,
 3,
 0,
 0,
 2,
 1,
 0,
 2,
 1,
 1,
 0,
 0,
 1,
 3,
 4,
 3,
 2,
 5,
 0,
 3,
 3,
 0,
 2,
 2,
 1,
 2,
 0,


In [39]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

# Normalize the data
X = X/255.0

model = Sequential()

model.add(Conv2D(256, (3, 3), input_shape=(3,60,60)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

model.add(Dense(64))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X, y, batch_size=32, epochs=10, validation_split=0.3)

ValueError: Negative dimension size caused by subtracting 2 from 1 for 'max_pooling2d_4/MaxPool' (op: 'MaxPool') with input shapes: [?,1,58,256].